## Will use MobileNet's frozen layers for feature extraction and then classify..

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.utils import image_dataset_from_directory

In [25]:
train_ds = image_dataset_from_directory(directory = 'dataset/train', label_mode='categorical', image_size=(224,224) ,shuffle=True)

Found 2481 files belonging to 432 classes.


In [24]:
test_ds = image_dataset_from_directory(directory = 'dataset/val', label_mode='categorical', image_size=(224,224) ,shuffle=False)

Found 851 files belonging to 432 classes.


In [26]:
len(train_ds.class_names)

432

In [27]:
for x,y in train_ds:
    print(f'Exact size of input images : {x.shape}') # (batch_size,x,y,channels)
    break

Exact size of input images : (32, 224, 224, 3)


In [31]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [32]:
base_model = MobileNetV2(    input_shape=(224,224,3), # this is the input tensor here, so no layers.Input() in seq. model
    alpha=1.0,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
    name=None,
)

In [33]:
base_model.trainable = False

In [43]:
model = Sequential([
    
    layers.Lambda(preprocess_input),
    base_model,

    layers.GlobalAveragePooling2D(),
    
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(432, activation='softmax')
])

In [44]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape              ┃        Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ lambda_2 (Lambda)                  │ ?                         │    0 (unbuilt) │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ mobilenetv2_1.00_224 (Functional)  │ (None, 7, 7, 1280)        │      2,257,984 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ global_average_pooling2d_1         │ ?                         │              0 │
│ (GlobalAveragePooling2D)           │                           │                │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense_4 (Dense)                    │ ?                         │    0 (unbuilt) │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense_5 (Dense)                    │ ?                         │    0 (unbuilt) │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense_6 (Dense)                    │ ?                         │    0 (unbuilt) │
└────────────────────────────────────┴───────────────────────────┴────────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [45]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy', tf.keras.metrics.Recall(name="recall")])

In [ ]:
model.fit(train_ds, batch_size=32, epochs=20, validation_data=test_ds)

Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 36s 421ms/step - accuracy: 0.0012 - loss: 6.1221 - recall: 0.0000e+00 - val_accuracy: 0.0235 - val_loss: 5.9373 - val_recall: 0.0000e+00
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 37s 369ms/step - accuracy: 0.0214 - loss: 5.4842 - recall: 0.0000e+00 - val_accuracy: 0.0341 - val_loss: 5.1575 - val_recall: 0.0000e+00
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 32s 411ms/step - accuracy: 0.0746 - loss: 4.3912 - recall: 0.0000e+00 - val_accuracy: 0.0635 - val_loss: 4.6395 - val_recall: 0.0012
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 29s 368ms/step - accuracy: 0.1467 - loss: 3.5927 - recall: 0.0073 - val_accuracy: 0.0764 - val_loss: 4.3857 - val_recall: 0.0082
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 30s 381ms/step - accuracy: 0.2479 - loss: 2.9582 - recall: 0.0371 - val_accuracy: 0.1046 - val_loss: 4.2663 - val_recall: 0.0176
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 30s 382ms/step - accuracy: 0.3519 - loss: 2.4470 - recall: 0.0943 - val_accuracy: 0.1234 - val_loss: 4.3060